In [1]:
from pydantic import BaseModel, ValidationError, Field, create_model
from typing import List, Dict, Any
import re


def validate_function_call(model_output: str, function_definitions: List[Dict[str, Any]]):
    result = {"model_output": model_output, "validation_result": ""}
    
    # Step 1: Check if the model output is in the correct format with square brackets
    if not re.match(r"^\[.*\]$", model_output.strip()):
        result["validation_result"] = "Invalid: The model output is not enclosed in square brackets."
        return result
    
    # Step 2: Extract the function call and parameters
    function_match = re.match(r"\[(\w+)\((.*)\)\]", model_output.strip())
    if not function_match:
        result["validation_result"] = "Invalid: The function call is not in the correct format."
        return result

    function_name = function_match.group(1)
    params_string = function_match.group(2)

    # Step 3: Parse the parameters into a dictionary
    try:
        params = dict(item.split('=') for item in params_string.split(', '))
    except ValueError:
        result["validation_result"] = "Invalid: The parameters are not in the correct key=value format."
        return result

    # Step 4: Find the matching function definition
    function_def = next((f for f in function_definitions if f['name'] == function_name), None)
    if not function_def:
        result["validation_result"] = f"Invalid: No matching function definition found for '{function_name}'."
        return result

    # Step 5: Dynamically create a validation model based on the function definition
    try:
        # Build dynamic fields based on the function definition
        fields = {}
        for param_name, param_details in function_def['parameters']['properties'].items():
            field_type = int if param_details['type'] == 'integer' else str  # Extend as needed for more types
            fields[param_name] = (field_type, Field(default=None) if param_name not in function_def['parameters'].get('required', []) else ...)

        # Dynamically create the model
        FunctionModel = create_model(function_name, **fields)

        # Step 6: Convert parameter values to expected types
        for key in params:
            try:
                param_type = fields[key][0]
                params[key] = param_type(params[key])  # Attempt type conversion
            except ValueError as e:
                result["validation_result"] = f"Invalid: Type mismatch for parameter '{key}'. Expected {param_type.__name__} but got '{params[key]}'. Error: {str(e)}"
                return result

        # Step 7: Validate the parameters using the dynamically created model
        FunctionModel(**params)
        result["validation_result"] = "Valid: The function call respects the parameter types and requirements."
        return result

    except ValidationError as e:
        result["validation_result"] = f"Invalid: {e.errors()}"
        return result
    except Exception as e:
        result["validation_result"] = f"Invalid: {str(e)}"
        return result


In [9]:
test_entry ={"id": "multiple_165", "question": [[{"role": "user", "content": "Find the price of a used Gibson Les Paul guitar in excellent condition in the Chicago area."}]], "function": [{"name": "identify_color_rgb", "description": "This function identifies the RGB values of a named color.", "parameters": {"type": "dict", "properties": {"color_name": {"type": "string", "description": "Name of the color."}, "standard": {"type": "string", "description": "The color standard (e.g. basic, pantone). Default is 'basic'"}}, "required": ["color_name"]}}, {"name": "board_game.chess.get_top_players", "description": "Find top chess players in a location based on rating.", "parameters": {"type": "dict", "properties": {"location": {"type": "string", "description": "The city you want to find the players from."}, "minimum_rating": {"type": "integer", "description": "Minimum rating to filter the players."}, "number_of_players": {"type": "integer", "default": 10, "description": "Number of players you want to retrieve, default value is 10"}}, "required": ["location", "minimum_rating"]}}, {"name": "guitar_price.find", "description": "Retrieve the price of a specific used guitar model based on its condition and location.", "parameters": {"type": "dict", "properties": {"model": {"type": "string", "description": "The model of the guitar."}, "condition": {"type": "string", "enum": ["Poor", "Good", "Excellent"], "description": "The condition of the guitar."}, "location": {"type": "string", "description": "The location where the guitar is being sold."}}, "required": ["model", "condition", "location"]}}]}


system_prompt_generate_test = """You are tasked with generating a similar question based on the provided input. Additionally, you are provided a function definition to help understand the context or processing involved. Follow these rules:

1. **Input Question Identification**:
   - Identify the type of question (e.g., mathematical, logical, geometry, etc.).
   - Preserve the same problem type in the output question (e.g., if the input is about calculating a factorial, the output must also be about factorials).

2. **Generate Similar Questions**:
   - Replace numeric values with different numbers while maintaining the same structure and context.
   - Use the function definition as context to ensure the generated question aligns with the problem type but do not call the function.

3. **Examples**:

   **Example 1**:
   Input Question: "Calculate the factorial of 2 using math functions."
   Function: `math.factorial`
   Output Question: "Calculate the factorial of 7 using math functions."

   **Example 2**:
   Input Question: "Determine if 27 is a prime number."
   Function: `math.is_prime`
   Output Question: "Determine if 43 is a prime number."

   **Example 3**:
   Input Question: "Find the area of a rectangle with a width of 8 units and length of 6 units."
   Function: None
   Output Question: "Find the area of a rectangle with a width of 10 units and length of 12 units."

Your task:
1. Use the input question and function definition as context.
2. Generate a similar question without calling the function.”"""



system_prompt_caller = """You are an expert in composing functions. You are given a question and a set of possible functions. Based on the question, you will need to make one or more function/tool calls to achieve the purpose.
If none of the function can be used, point it out. If the given question lacks the parameters required by the function, also point it out.
You should only return the function calls in your response.

If you decide to invoke any of the function(s), you MUST put it in the format of [func_name1(params_name1=params_value1, params_name2=params_value2...), func_name2(params)]
You SHOULD NOT include any other text in the response.

Here is a list of functions in JSON format that you can invoke: \n{functions}\n

"""

system_prompt_tester_caller ="""You are an expert in composing functions. You are given a question and a set of possible functions. Based on the question, you will need to make one or more function/tool calls to achieve the purpose.

If none of the functions can be used, point it out. If the given question lacks the parameters required by the function, also point it out.

You should only return the function calls in your response, prefixed with the phrase **"Validate this function call:"**. For example:
`Validate this function call: [func_name1(params_name1=params_value1, params_name2=params_value2...)]`

If you decide to invoke any of the function(s), you MUST put it in the format:
`Validate this function call: [func_name1(params_name1=params_value1, params_name2=params_value2...), func_name2(params)]`

You SHOULD NOT include any other text in the response.

Here is a list of functions in JSON format that you can invoke: \n{functions}\n"""

system_prompt_caller = system_prompt_caller.format(functions=test_entry["function"])
system_prompt_tester_caller = system_prompt_tester_caller.format(functions=test_entry["function"])

print(system_prompt_caller)
# test_entry => "["question"][0][1]['content']" => system_prompt_generate_test, function_list => system_prompt_caller => bfcl_eval => feedback, function_list,"question[0][0]" => system_prompt_caller => result  => validator (we can cycle) => final_result

You are an expert in composing functions. You are given a question and a set of possible functions. Based on the question, you will need to make one or more function/tool calls to achieve the purpose.
If none of the function can be used, point it out. If the given question lacks the parameters required by the function, also point it out.
You should only return the function calls in your response.

If you decide to invoke any of the function(s), you MUST put it in the format of [func_name1(params_name1=params_value1, params_name2=params_value2...), func_name2(params)]
You SHOULD NOT include any other text in the response.

Here is a list of functions in JSON format that you can invoke: 
[{'name': 'identify_color_rgb', 'description': 'This function identifies the RGB values of a named color.', 'parameters': {'type': 'dict', 'properties': {'color_name': {'type': 'string', 'description': 'Name of the color.'}, 'standard': {'type': 'string', 'description': "The color standard (e.g. basic, p

In [ ]:
from agent.agent_framework import OpenSourceAgent, Mediator
model_name_bfcl = 'Qwen/Qwen2.5-1.5B-Instruct'  # Sostituisci con il nome effettivo del modello
temperature = 0.9  # Sostituisci con il valore desiderato per la temperatura

tester = OpenSourceAgent(name="tester", system_instruction=system_prompt_generate_test,model_name = model_name_bfcl)
caller = OpenSourceAgent(name="caller", system_instruction=system_prompt_caller,model_name = model_name_bfcl)
mediator = Mediator()
mediator.add_agent(tester)
mediator.add_agent(caller)


print('USER:                    ', test_entry["question"][0][0]['content'])
res_test = mediator.send("user",tester.id, test_entry["question"][0][0]['content'])
print('AGENT-TESTER:            ', res_test)

res_call = mediator.send(tester.id,caller.id, res_test)
print('AGENT-CALLER:            ', res_call)

validator =validate_function_call(res_call, test_entry["function"])
print('VALIDATOR:               ', validator)

system_prompt_caller_feedback_skeleton = """
You are an expert in refining function calls. Your task is to analyze the user's question, the previous function call, and the validator feedback to produce a corrected function call.

Guidelines:
1. Analyze the user's question.
2. Review the model's previous output: {model_output}.
3. Address the issues described in the validation feedback: {validation_result}.
4. Use the list of available functions: {functions}.

Identify and fix issues such as:
   - Missing fields.
   - Incorrect parameter values or types.
   - Any other validation feedback.

If the question cannot be answered with the available functions, explicitly note this in the function call response.

### Output Format
- [func_name(param1=value1, param2=value2, ...)]

### Example:
**Question**: "Search for items with a price below $100."
**Model Output**: [search_items(price_limit=100)]
**Validation Feedback**: 
  - The field 'category' is required but was missing in the function call.
  - Add the 'category' parameter to specify the type of items (e.g., 'electronics', 'clothing').

### Corrected Output:
[search_items(price_limit=100, category='electronics')]
"""

system_prompt_caller_feedback = system_prompt_caller_feedback_skeleton.format(functions= test_entry["function"],model_output = res_call, validation_result = validator["validation_result"])

caller_feedback = OpenSourceAgent(name="caller_feedback", system_instruction=system_prompt_caller_feedback,model_name = model_name_bfcl)
mediator.add_agent(caller_feedback)



res_final = mediator.send("user",caller_feedback.id, test_entry["question"][0][0]['content'])

print('AGENT-CALLER-FEEDBACK:   ', res_final)

validator =validate_function_call(res_final, test_entry["function"])
print('VALIDATOR:               ', validator)

system_prompt_validator = """
You are an expert validator ensuring function calls meet user requirements.

Your task:
1. Analyze the user question, function call, and validation feedback.
2. Identify and explain:
   - Missing fields or parameters.
   - Incorrect values or types.
   - Misalignment with the user's intent.
3. Provide actionable corrections.

### Required Output:
- **Question**: The original user question.
- **Function Calling Response**: The model-generated function call.
- **Validation Feedback**:
  - Clearly state what is missing, incorrect, or misaligned.
  - Provide precise steps to fix the issues.

### Example:
**Question**: {model_question}
**Function Calling Response**: {model_output}
**Validation Feedback**: {validation_result}

Be concise and actionable in your feedback.
"""
system_prompt_validator = system_prompt_validator.format(validation_result = validator["validation_result"], model_question = test_entry["question"][0][0]['content'], model_output = res_final)

validator_agent = OpenSourceAgent(name="validator_agent", system_instruction=system_prompt_validator,model_name = model_name_bfcl)
mediator.add_agent(validator_agent)

caller_feedback_content = '\n **Question**: ' + test_entry["question"][0][0]['content'] + '\n **function calling response**: ' + res_final + '\n **validation feedback**: ' + validator["validation_result"]
res_validator = mediator.send(caller_feedback.id,validator_agent.id, caller_feedback_content)
print('AGENT-VALIDATOR:         ', res_validator)


final_system_prompt = """You are an expert in composing function calls. You will be provided with:
1. A user question.
2. A previous model output (function call).
3. Feedback from a validator with detailed guidance on how to fix the function call.

Your task is to:
- Analyze the question, incorrect model output, and validator feedback.
- Generate a corrected function call that addresses all issues raised in the feedback while fulfilling the purpose of the user question.

You MUST return the function call in the format:
[func_name(param1=value1, param2=value2, ...)]

DO NOT include any additional text in your response.

### Example Input:
Question: "Find a gluten-free recipe that can be made in under 30 minutes."

Your function calling response: [recipe_search(diet=['Gluten Free'], time_limit=29)]

Validation Feedback: The field 'dish' is required but was missing in the function call. Add the 'dish' parameter to specify the type of recipe to search for (e.g., 'dessert', 'main course'). Ensure all required fields are included in the function call.

### Example Output:
[recipe_search(diet=['Gluten Free'], time_limit=29, dish='main course')]"""

final_caller = OpenSourceAgent(name="final_caller", system_instruction=final_system_prompt,model_name = model_name_bfcl)
mediator.add_agent(final_caller)

print('--------------VALIDATOR-CHAT--------------')

#res_super_final = mediator.chat(sender_id=validator_agent
#.id,recipient_id=final_caller.id,content=res_validator,str_condition="Valid: The function call respects the parameter types and requirements.",max_turns=4)

counter = 0
max_turns = 4
res_recipient = res_validator
str_condition = "Valid: The function call respects the parameter types and requirements."
end = False

while counter < max_turns and not end:

   res_recipient = mediator.send(validator_agent.id, final_caller.id, res_recipient if counter == 0 else res_sender)

   print(f'[{final_caller.name}]', res_recipient)
   if str_condition and str_condition in res_recipient:
      break

   if callable(getattr(final_caller, 'end_condition', False)):
      end = final_caller.end_condition()

   if end:
      break

   res_super_final = res_recipient

   validator = validate_function_call(res_recipient, test_entry["function"])
   print('VALIDATOR:               ', validator)
   caller_feedback_content = '\n **Question**: ' + test_entry["question"][0][0]['content'] + '\n **function calling response**: ' + res_final + '\n **validation feedback**: ' + validator["validation_result"]

   res_sender = mediator.send(final_caller.id, validator_agent.id, caller_feedback_content)
   print(f'[{validator_agent.name}]', res_sender)
   if str_condition and str_condition in res_sender:
      break

   if callable(getattr(validator_agent, 'end_condition', False)):
      end = validator_agent.end_condition()

   counter += 1


print('------------------------------------------')
print('AGENT-CALLER-FEEDBACK-V: ', res_super_final)

res_without_agent = mediator.send("user",caller.id, test_entry["question"][0][0]['content'])
print('AGENT-CALLER-FEEDBACK:   ', res_final)
print('AGENT-QUANTIZE:          ', res_without_agent)

USER:                     Find the price of a used Gibson Les Paul guitar in excellent condition in the Chicago area.
AGENT-TESTER:             Determine the price of a used Gibson Les Paul guitar in excellent condition in the New York area.
AGENT-CALLER:             [guitar_price.find(model="Les Paul", condition="Excellent", location="New York")]
VALIDATOR:                {'model_output': '[guitar_price.find(model="Les Paul", condition="Excellent", location="New York")]', 'validation_result': 'Invalid: The function call is not in the correct format.'}
AGENT-CALLER-FEEDBACK:    [guitar_price.find(model="Gibson Les Paul", condition="Excellent", location="Chicago")]
VALIDATOR:                {'model_output': '[guitar_price.find(model="Gibson Les Paul", condition="Excellent", location="Chicago")]', 'validation_result': 'Invalid: The function call is not in the correct format.'}
AGENT-VALIDATOR:          **Question**: Find the price of a used Gibson Les Paul guitar in excellent condition i